# Задание 1

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, f1_score, recall_score, precision_score
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

import sys
sys.path.append(r"D:\university\ML\MyLib")
import classification_metrics as cm


In [2]:
#Какова вероятность отправиться на прогулку если идёт дождь, при наличии следующих наблюдений?
data = [
        ('солнечно', True),
        ('снег', False),
        ('облачно', False),
        ('дождь', False),
        ('солнечно', True),
        ('снег', False),
        ('облачно', True),
        ('снег', False),
        ('солнечно', False),
        ('облачно', True),
        ('снег', True),
        ('солнечно', True),
        ('дождь', False),
        ('дождь', True),
        ('облачно', True),
]

In [3]:
df = pd.DataFrame(data, columns=['weather', 'status'])
df

,weather,status
0,солнечно,True
1,снег,False
2,облачно,False
3,дождь,False
4,солнечно,True
5,снег,False
6,облачно,True
7,снег,False
8,солнечно,False
9,облачно,True


In [4]:
len(df.loc[(df['weather'] == 'дождь') & (df['status'] == True)]) / len(df.loc[df['weather'] == 'дождь'])

0.3333333333333333

# Задание 2

In [5]:
df = pd.read_csv('../data/card_transdata.csv')
df.head(10)

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,57.877857,0.311140,1.945940,1.0,1.0,0.0,0.0,0.0
1,10.829943,0.175592,1.294219,1.0,0.0,0.0,0.0,0.0
2,5.091079,0.805153,0.427715,1.0,0.0,0.0,1.0,0.0
3,2.247564,5.600044,0.362663,1.0,1.0,0.0,1.0,0.0
4,44.190936,0.566486,2.222767,1.0,1.0,0.0,1.0,0.0
5,5.586408,13.261073,0.064768,1.0,0.0,0.0,0.0,0.0
6,3.724019,0.956838,0.278465,1.0,0.0,0.0,1.0,0.0
7,4.848247,0.320735,1.273050,1.0,0.0,1.0,0.0,0.0
8,0.876632,2.503609,1.516999,0.0,0.0,0.0,0.0,0.0
9,8.839047,2.970512,2.361683,1.0,0.0,0.0,1.0,0.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 8 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   distance_from_home              1000000 non-null  float64
 1   distance_from_last_transaction  1000000 non-null  float64
 2   ratio_to_median_purchase_price  1000000 non-null  float64
 3   repeat_retailer                 1000000 non-null  float64
 4   used_chip                       1000000 non-null  float64
 5   used_pin_number                 1000000 non-null  float64
 6   online_order                    1000000 non-null  float64
 7   fraud                           1000000 non-null  float64
dtypes: float64(8)
memory usage: 61.0 MB


In [7]:
df['used_chip'] = df['used_chip'].astype(int)
df['repeat_retailer'] = df['repeat_retailer'].astype(int)
df['used_pin_number'] = df['used_pin_number'].astype(int)
df['online_order'] = df['online_order'].astype(int)
df['fraud'] = df['fraud'].astype(int)

In [8]:
outlier=df[['distance_from_home',	'distance_from_last_transaction',	'ratio_to_median_purchase_price']]
Q1 = outlier.quantile(0.25)
Q3 = outlier.quantile(0.75)
IQR = Q3-Q1
df_filtered = outlier[~((outlier < (Q1 - 3 * IQR)) |(outlier > (Q3 + 3 * IQR))).any(axis=1)]

In [9]:
X = df.drop(['fraud'], axis=1)
y= df['fraud']

In [10]:
X

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order
0,57.877857,0.311140,1.945940,1,1,0,0
1,10.829943,0.175592,1.294219,1,0,0,0
2,5.091079,0.805153,0.427715,1,0,0,1
3,2.247564,5.600044,0.362663,1,1,0,1
4,44.190936,0.566486,2.222767,1,1,0,1
...,...,...,...,...,...,...,...
999995,2.207101,0.112651,1.626798,1,1,0,0
999996,19.872726,2.683904,2.778303,1,1,0,0
999997,2.914857,1.472687,0.218075,1,1,0,1
999998,4.258729,0.242023,0.475822,1,0,0,1


In [11]:
y

0         0
1         0
2         0
3         0
4         0
         ..
999995    0
999996    0
999997    0
999998    0
999999    0
Name: fraud, Length: 1000000, dtype: int32

In [12]:
#scaler = MinMaxScaler(feature_range=(0, 1))
#X = scaler.fit_transform(X)

## KNN

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

In [14]:
knn4 = KNeighborsClassifier(n_neighbors=4)
knn5 = KNeighborsClassifier(n_neighbors=5)

In [15]:
pipe_knn3 = Pipeline(
    [('minmaxscaler', MinMaxScaler()), ('model', KNeighborsClassifier(n_neighbors=3)) 
    ]
)
pipe_knn3.fit(X_train, y_train)

pipe_knn4 = Pipeline(
    [('minmaxscaler', MinMaxScaler()), ('model', KNeighborsClassifier(n_neighbors=4)) 
    ]
)
pipe_knn4.fit(X_train, y_train)

pipe_knn5 = Pipeline(
    [('minmaxscaler', MinMaxScaler()), ('model', KNeighborsClassifier(n_neighbors=5)) 
    ]
)
pipe_knn5.fit(X_train, y_train)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('model', KNeighborsClassifier())])

In [16]:
y_knn3_pred = pipe_knn3.predict(X_test)
y_knn4_pred = pipe_knn4.predict(X_test)
y_knn5_pred = pipe_knn5.predict(X_test)

In [17]:
confusion_matrix(y_test, y_knn3_pred)

array([[182462,     57],
       [   122,  17359]], dtype=int64)

In [18]:
print(f"Accuracy: {accuracy_score(y_test, y_knn3_pred)}")
print(f"ROC AUC score: {roc_auc_score(y_test, y_knn3_pred)}")
print(f"F1 score: {f1_score(y_test, y_knn3_pred)}")
print(f"Recall score: {recall_score(y_test, y_knn3_pred)}")
print(f"Precision score: {precision_score(y_test, y_knn3_pred)}")

Accuracy: 0.999105
ROC AUC score: 0.996354348984105
F1 score: 0.9948706192509384
Recall score: 0.9930209942222985
Precision score: 0.9967271474506201


In [19]:
confusion_matrix(y_test, y_knn4_pred)

array([[182499,     20],
       [   197,  17284]], dtype=int64)

In [20]:
print(f"Accuracy: {accuracy_score(y_test, y_knn4_pred)}")
print(f"ROC AUC score: {roc_auc_score(y_test, y_knn4_pred)}")
print(f"F1 score: {f1_score(y_test, y_knn4_pred)}")
print(f"Recall score: {recall_score(y_test, y_knn4_pred)}")
print(f"Precision score: {precision_score(y_test, y_knn4_pred)}")

Accuracy: 0.998915
ROC AUC score: 0.9943105220924802
F1 score: 0.9937616788845767
Recall score: 0.9887306218179738
Precision score: 0.9988441978733241


In [21]:
confusion_matrix(y_test, y_knn5_pred)

array([[182467,     52],
       [   134,  17347]], dtype=int64)

In [22]:
print(f"Accuracy: {accuracy_score(y_test, y_knn5_pred)}")
print(f"ROC AUC score: {roc_auc_score(y_test, y_knn5_pred)}")
print(f"F1 score: {f1_score(y_test, y_knn5_pred)}")
print(f"Recall score: {recall_score(y_test, y_knn5_pred)}")
print(f"Precision score: {precision_score(y_test, y_knn5_pred)}")

Accuracy: 0.99907
ROC AUC score: 0.9960248163958857
F1 score: 0.9946674311926607
Recall score: 0.9923345346376066
Precision score: 0.9970113224897983


## Naive Bayes

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

In [24]:
nb = GaussianNB()

nb.fit(X_train, y_train)
y_pred=nb.predict(X_test)

confusion_matrix(y_test, y_pred)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"ROC AUC score: {roc_auc_score(y_test, y_pred)}")
print(f"F1 score: {f1_score(y_test, y_pred)}")
print(f"Recall score: {recall_score(y_test, y_pred)}")
print(f"Precision score: {precision_score(y_test, y_pred)}")

Accuracy: 0.95105
ROC AUC score: 0.7911050595540128
F1 score: 0.6808163797600416
Recall score: 0.5972770436473885
Precision score: 0.791524524296869


In [25]:
confusion_matrix(y_test, y_pred)

array([[179769,   2750],
       [  7040,  10441]], dtype=int64)

In [26]:
pipe = Pipeline(
    [('minmaxscaler', MinMaxScaler()), ('model', GaussianNB()) 
    ]
)
pipe.fit(X_train, y_train)

y_nb_pred=pipe.predict(X_test)

confusion_matrix(y_test, y_nb_pred)
print(f"Accuracy: {accuracy_score(y_test, y_nb_pred)}")
print(f"ROC AUC score: {roc_auc_score(y_test, y_nb_pred)}")
print(f"F1 score: {f1_score(y_test, y_nb_pred)}")
print(f"Recall score: {recall_score(y_test, y_nb_pred)}")
print(f"Precision score: {precision_score(y_test, y_nb_pred)}")

Accuracy: 0.951065
ROC AUC score: 0.791165003960229
F1 score: 0.6809245916604181
Recall score: 0.5973914535781706
Precision score: 0.7916161309884778


In [27]:
confusion_matrix(y_test, y_nb_pred)

array([[179770,   2749],
       [  7038,  10443]], dtype=int64)

## Logistic Regression

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

In [29]:
lr = LogisticRegression().fit(X_train, y_train)
y_pred = lr.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"ROC AUC score: {roc_auc_score(y_test, y_pred)}")
print(f"F1 score: {f1_score(y_test, y_pred)}")
print(f"Recall score: {recall_score(y_test, y_pred)}")
print(f"Precision score: {precision_score(y_test, y_pred)}")

Accuracy: 0.959855
ROC AUC score: 0.8053433620881723
F1 score: 0.7291069199365701
Recall score: 0.6180996510497111
Precision score: 0.8887152492186214


D:\university\ML\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [30]:
confusion_matrix(y_test, y_pred)

array([[181166,   1353],
       [  6676,  10805]], dtype=int64)

In [31]:
pipe = Pipeline(
    [('minmaxscaler', MinMaxScaler()), ('model', LogisticRegression()) 
    ]
)
pipe.fit(X_train, y_train)

y_lr_pred=pipe.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_lr_pred)}")
print(f"ROC AUC score: {roc_auc_score(y_test, y_lr_pred)}")
print(f"F1 score: {f1_score(y_test, y_lr_pred)}")
print(f"Recall score: {recall_score(y_test, y_lr_pred)}")
print(f"Precision score: {precision_score(y_test, y_lr_pred)}")

Accuracy: 0.944485
ROC AUC score: 0.7005307639409974
F1 score: 0.5604339047468229
Recall score: 0.40489674503746925
Precision score: 0.9100025713551041


In [32]:
confusion_matrix(y_test, y_lr_pred)

array([[181819,    700],
       [ 10403,   7078]], dtype=int64)

## SVM

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

In [34]:
svml = LinearSVC(max_iter=10000).fit(X_train, y_train)

y_pred = svml.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"ROC AUC score: {roc_auc_score(y_test, y_pred)}")
print(f"F1 score: {f1_score(y_test, y_pred)}")
print(f"Recall score: {recall_score(y_test, y_pred)}")
print(f"Precision score: {precision_score(y_test, y_pred)}")

D:\university\ML\venv\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Accuracy: 0.94844
ROC AUC score: 0.7387766050427126
F1 score: 0.621689045417859
Recall score: 0.48469767175790857
Precision score: 0.8666257543213665


D:\university\ML\venv\lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [35]:
confusion_matrix(y_test, y_pred)

array([[181215,   1304],
       [  9008,   8473]], dtype=int64)

In [36]:
pipe = Pipeline(
    [('minmaxscaler', MinMaxScaler()), ('model', LinearSVC(max_iter=10000)) 
    ]
)
pipe.fit(X_train, y_train)

y_svml_pred=pipe.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_svml_pred)}")
print(f"ROC AUC score: {roc_auc_score(y_test, y_svml_pred)}")
print(f"F1 score: {f1_score(y_test, y_svml_pred)}")
print(f"Recall score: {recall_score(y_test, y_svml_pred)}")
print(f"Precision score: {precision_score(y_test, y_svml_pred)}")

D:\university\ML\venv\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Accuracy: 0.94152
ROC AUC score: 0.6853540415916082
F1 score: 0.5284631511046607
Recall score: 0.3749213431725874
Precision score: 0.8949883927352178


In [63]:
print(f"Accuracy: {cm.accuracy(y_test, y_svml_pred)}")
print(f"F1 score: {cm.f1(y_test, y_svml_pred)}")
print(f"Recall score: {cm.recall(y_test, y_svml_pred)}")
print(f"Precision score: {cm.precision(y_test, y_svml_pred)}")

Accuracy: 0.94152
F1 score: 0.5284631511046607
Recall score: 0.3749213431725874
Precision score: 0.8949883927352178


In [51]:
confusion_matrix(y_test, y_svml_pred)

array([[181750,    769],
       [ 10927,   6554]], dtype=int64)

## Classifiers

In [64]:
classifiers = {
    'Linear SVC': LinearSVC(max_iter=10000),
    'KNN classifier': KNeighborsClassifier(n_neighbors=3),
    'Naive Bayes': GaussianNB(),
    'Log reg': LogisticRegression(),
}

n_classifiers = len(classifiers)

for index, (name, classifier) in enumerate(classifiers.items()):
    classifier.fit(X_train, np.ravel(y_train))

    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy (train) for %s: %0.1f%% " % (name, accuracy * 100))
    print(classification_report(y_test,y_pred))

D:\university\ML\venv\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
D:\university\ML\venv\lib\site-packages\sklearn\svm\_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Accuracy (train) for Linear SVC: 94.6% 
              precision    recall  f1-score   support

           0       0.95      0.99      0.97    182519
           1       0.89      0.43      0.58     17481

    accuracy                           0.95    200000
   macro avg       0.92      0.71      0.78    200000
weighted avg       0.94      0.95      0.94    200000

Accuracy (train) for KNN classifier: 98.5% 
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    182519
           1       0.90      0.93      0.92     17481

    accuracy                           0.99    200000
   macro avg       0.95      0.96      0.95    200000
weighted avg       0.99      0.99      0.99    200000

Accuracy (train) for Naive Bayes: 95.0% 
              precision    recall  f1-score   support

           0       0.96      0.98      0.97    182519
           1       0.79      0.59      0.68     17481

    accuracy                           0.95    200000
   

D:\university\ML\venv\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Задание 3

In [62]:
import numpy as np


def accuracy(y_test, y_pred):
    right = 0
    y_test = np.array(y_test)
    for i in range(len(y_test)):
        if(y_test[i] == y_pred[i]):
            right += 1
    return right/len(y_test)


def precision(y_test, y_pred):
    right_pred = 0
    y_test = np.array(y_test)
    for i in range(len(y_pred)):
        if (y_test[i] == y_pred[i] == 1):
            right_pred += 1
    positive = list(filter(lambda x: x == 1, y_pred))
    return right_pred / len(positive)


def recall(y_test, y_pred):
    right_pred = 0
    y_test = np.array(y_test)
    tp_fn = 0
    for i in range(len(y_pred)):
        if (y_test[i] == y_pred[i] == y_pred[i] == 1):
            right_pred += 1
        if (y_test[i] == 1):
            tp_fn += 1
    return right_pred / tp_fn


def f1(y_test, y_pred):
    pr = precision(y_test, y_pred)
    rec = recall(y_test, y_pred)
    return 2 * pr * rec / (pr + rec)
